In [321]:
import numpy as np
import tensorflow as tf
from copy import deepcopy

import os

import datetime

import import_ipynb
from QBert import train_utils, models

from tqdm.notebook import tqdm



In [322]:
# sub_model (core_model 필요 Config)

vocab_size = 32000 # 
hidden_size = 768 # Transformer hidden Layers
type_vocab_size = 12 #: The number of types that the 'type_ids' input can take.
num_layers = 12
num_attention_heads = 12
max_seq_length = 256 # 512
dropout_rate = .1
# attention_dropout_rate = .1
inner_dim = 3072
# hidden_act = 'gelu'
initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)

# Pretrain Model 필요 Config
max_predictions_per_seq = 40


In [323]:
filenames = ['./Test_Examples2.tfrecords' ]

# Create a description of the features.
feature_description = {
    'input_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'input_mask': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([max_predictions_per_seq], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}

# keys = feature_description.keys()

In [324]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

def _select_data_from_record(record):
    """Filter out features to use for pretraining."""
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids'],
        'masked_lm_positions': record['masked_lm_positions'],
        'masked_lm_ids': record['masked_lm_ids'],
        'masked_lm_weights': record['masked_lm_weights'],
    }
    if use_next_sentence_label:
        x['next_sentence_labels'] = record['next_sentence_labels']
    if use_position_id:
        x['position_ids'] = record['position_ids']

    # TODO(hongkuny): Remove the fake labels after migrating bert pretraining.
    if output_fake_labels:
        return (x, record['masked_lm_weights'])
    else:
        return x


In [325]:

BUFFER_SIZE = 10000

GLOBAL_BATCH_SIZE = 4
#BATCH_SIZE_PER_REPLICA = np.ceil(GLOBAL_BATCH_SIZE // strategy.num_replicas_in_sync)

use_next_sentence_label = True
output_fake_labels = True
use_position_id = False


In [326]:


train_dataset = tf.data.Dataset.from_tensor_slices(filenames)
train_dataset = train_dataset.interleave(tf.data.TFRecordDataset, cycle_length = -1)

dataset_inputs = train_dataset.map(_parse_function) # String to Example
dataset_inputs_with_labels = dataset_inputs.map(_select_data_from_record) # Example to InputData
## 본래대로라면 그냥 써도 되지만, 현재 Label이 없는 데이터이기 때문에
## max_predictions_per_seq 길이의 허위 정답 (Fake_y)를 삽입하는 mapping function이다.

dataset = dataset_inputs_with_labels
dataset = dataset.cache()
dataset = dataset.shuffle(10000, reshuffle_each_iteration = True)
dataset = dataset.batch(GLOBAL_BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)



In [330]:
from collections import Counter

In [333]:
tf.reshape(data['masked_lm_ids'], (-1)).numpy()

array([ 3130,  2357,  3052,  6005,   738, 24311,  3113, 13649,  3191,
       13370,    17, 31622,  5741,  3091, 11627,  3209,  5777,  5579,
        2987,  6275,  4189,  6007,  8667, 29452,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,    17, 24783,    15,  6997,  5621,
       11406,  7612,    15,  9892, 28545,  9892,   170,  5795, 17950,
        9892,  8686,  1779,  3214,  7719,  1779,  3053,  7090, 14161,
         560, 13864, 19648,    15, 14883, 14883,  3121, 14883,  3130,
       17329,     0,     0,     0,     0,     0,     0,     0,  2009,
        5651,  3238, 20114, 16151,  5726, 17192,    17,  6731,  6879,
        5603,  3130,  9855, 10815,  9064,  1850, 21296,   178,  6192,
        4150,  6516, 24908,  6616,  1573,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  6703,  5584, 17898,  7471,    15,  5651,
        3130,  7070,

In [336]:
for i, datas in enumerate(dataset) :
    
    data, labels = datas
    
    word_count.update(Counter(tf.reshape(data['masked_lm_ids'], (-1)).numpy()))
    
    if i == 100 :
        break

In [352]:
for key, values in word_count.items():
    
    if values == 73 :
        print(key, values)
        break

12 73


In [348]:
sorted(word_count.values())[::-1]

[6927,
 436,
 293,
 93,
 73,
 68,
 64,
 63,
 57,
 53,
 49,
 43,
 41,
 39,
 37,
 36,
 35,
 34,
 33,
 32,
 30,
 28,
 28,
 27,
 26,
 24,
 23,
 21,
 21,
 21,
 19,
 18,
 18,
 17,
 17,
 16,
 16,
 16,
 16,
 16,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5

In [287]:
from transformers import BertTokenizerFast
tokenizer_for_load = BertTokenizerFast.from_pretrained('./model/BertTokenizer-3000-32000-vocab.txt'
                                                   , strip_accents=False
                                                   , lowercase=False)

Calling BertTokenizerFast.from_pretrained() with the path to a single file or url is deprecated


In [294]:
ds = next(iter(dataset))

In [295]:
inputs = ds[0]



In [296]:
ina = {}
for key in inputs.keys():
    ina[key] = inputs[key][3]

In [297]:
t_input = tokenizer_for_load.convert_ids_to_tokens(ina['input_ids'])
t_output = tokenizer_for_load.convert_ids_to_tokens(ina['masked_lm_ids'])


In [298]:
print(tokenizer_for_load.convert_ids_to_tokens(ina['input_ids']))

['[CLS]', '여름에', '열린', '고시엔', '대회의', '준', '##준', '##결승', '##전에서는', '하타', '##야마', '히토', '##시', ',', '미즈', '##노', '가쓰', '##히토', '[MASK]', '소속된', '[MASK]', '현립', '[MASK]', '고등학교', '##와', '대결', '##하였지만', '선발', '투수', '##인', '아라', '##키와', '구원', '투수', '##인', '이시', '##이가', '모두', '부진', '##하면서', '2대', '14', '##로', '크게', '패했다', '.', '학', '##년을', '늘', '##릴', '때마다', '고시엔', '대회에서', '성적이', '저하', '##했던', '것에', '대해', '아라', '##키는', '"', '1학년', '때에는', '힘이', '없었기', '때문에', '타자', '##의', '손', '[MASK]', '##림에서', '성장', '##하지', '[MASK]', '자연스럽게', '[MASK]', '##은', '공이', '3학년', '[MASK]', '그대로', '성장', '##해', '타자', '##에게', '있어서의', '치', '[MASK]', '좋은', '무렵', '##이', '되었다', '[MASK]', '라고', '분석', '##했다', '.', '프로', '입단', '후', '.', '1982년', '가을에', '[MASK]', '프로', '야구', '드래프트', '회의에서', '##는', '야쿠르트', '스', '##왈', '##로스', '##와', '[MASK]', '[MASK]', '##가', '[MASK]', '##순위로', '지명', '##하면서', '추첨', '결과', '야쿠르트', '##가', '교섭', '##권을', '획득했다', '[SEP]', '[MASK]', '인기를', '얻', '##으면서', '아라', '##키를', '둘러싸', '##는', '팬들의', '혼란을', '##소에

In [299]:
for i, m_p in enumerate(ina['masked_lm_positions']) :
    print("{} \t:{}\t vs\t{}".format(m_p,t_input[m_p], t_output[i]))

207 	:[MASK]	 vs	제1차
144 	:[MASK]	 vs	진
247 	:[MASK]	 vs	활약
129 	:[MASK]	 vs	절대적인
139 	:##소에	 vs	피하기
246 	:[MASK]	 vs	##서
79 	:[MASK]	 vs	때는
150 	:[MASK]	 vs	##와
22 	:[MASK]	 vs	이케다
117 	:[MASK]	 vs	1
214 	:[MASK]	 vs	상대
196 	:##슬링	 vs	맡았다
69 	:[MASK]	 vs	##놀
250 	:##인	 vs	##인
188 	:[MASK]	 vs	##테이션
168 	:여의	 vs	지금도
163 	:[MASK]	 vs	터널
181 	:[MASK]	 vs	3년
115 	:[MASK]	 vs	자이언츠
114 	:[MASK]	 vs	요미우리
209 	:[MASK]	 vs	선발
75 	:[MASK]	 vs	가라앉
161 	:[MASK]	 vs	아라
212 	:대동	 vs	10
205 	:[MASK]	 vs	올스타전
154 	:[MASK]	 vs	지하
19 	:소속된	 vs	소속된
20 	:[MASK]	 vs	도쿠시마
87 	:[MASK]	 vs	##기
147 	:[MASK]	 vs	##의
73 	:[MASK]	 vs	못하고
200 	:[MASK]	 vs	투표
80 	:그대로	 vs	그대로
227 	:[MASK]	 vs	전년도
92 	:[MASK]	 vs	"
18 	:[MASK]	 vs	등이
103 	:[MASK]	 vs	열린
156 	:[MASK]	 vs	만들어졌다
0 	:[CLS]	 vs	[PAD]
0 	:[CLS]	 vs	[PAD]


In [206]:
print(ina['masked_lm_positions'])

tf.Tensor(
[ 94 100  76 122  99  87  65  54  67  32  21  18   5 116  43  11 102   1
 108   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0], shape=(40,), dtype=int64)


In [205]:
print(tokenizer_for_load.convert_ids_to_tokens(ina['masked_lm_ids']))

['남부의', '하여', '전쟁', '.', '수도로', '##탱', '그리하여', '아르', '대성', '보냈다', '이때', '공방', '는', '##시', '##수', '이름', '프랑스', '"', '11월', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
print(tokenizer_for_load.convert_ids_to_tokens(ina['input_ids']))

In [317]:
# optimizer

## Learning Rate Decay

# lr = 1e-4 warmup stage (step <= 10000)
# Decay linearly

init_lr = 1e-4
warmup_steps = 10000
num_train_steps = 20 * 50000
end_lr = 0

# decay_schedule_fn = tf.keras.optimizers.schedules.PolynomialDecay(
#       initial_learning_rate=init_lr,
#       decay_steps=num_train_steps,
#       end_learning_rate=end_lr)

# lr_schedule = train_utils.WarmUp(
#         initial_learning_rate=init_lr,
#         decay_schedule_fn=decay_schedule_fn,
#         warmup_steps=warmup_steps)

# optimizer = train_utils.AdamWeightDecay( 
#     learning_rate=lr_schedule,
#     weight_decay_rate=0.01,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-6,
#     exclude_from_weight_decay=['LayerNorm', 'layer_norm', 'bias'])

optimizer = tf.keras.optimizers.Adam(lr = init_lr, beta_1 = 0.9, beta_2 = 0.999, epsilon=1e-6)

In [318]:
# loss
def loss_fn(fake_y, losses, **unused_args) :
    
    return tf.reduce_mean(losses, axis = -1)
    

In [319]:
model, sub_model = models.get_bert_models_fn(vocab_size
                                         , hidden_size
                                         , type_vocab_size
                                         , num_layers
                                         , num_attention_heads
                                         , max_seq_length
                                         , max_predictions_per_seq
                                         , dropout_rate
                                         , inner_dim 
                                         , initializer)
model.compile(optimizer, loss=loss_fn)

In [10]:
model.load_weights(filepath='./training_checkpoints_load/ckpt_1')

In [300]:
inputs, labels = next(iter(dataset))

In [301]:
sub_res = sub_model([inputs['input_ids'], inputs['input_mask'][:, tf.newaxis, tf.newaxis, :], inputs['segment_ids']])

In [302]:
layer_x = model.layers[6]

In [303]:
output_logits = layer_x([sub_res['sequence_output'], tf.cast(inputs['masked_lm_positions'], dtype = tf.int32)])

In [304]:
seg_output, masked_lm_positions = sub_res['sequence_output'], tf.cast(inputs['masked_lm_positions'], dtype = tf.int32)
        
sequence_shape = tf.shape(seg_output) # [batch_size, seq_length, dff]
batch_size, seq_length = sequence_shape[0], sequence_shape[1] 
width = seg_output.shape.as_list()[2] or sequence_shape[2]

flat_offsets = tf.reshape(
    tf.range(0, batch_size, dtype=tf.int32) * seq_length, [-1, 1])
flat_positions = tf.reshape(masked_lm_positions + flat_offsets, [-1])
flat_sequence_tensor = tf.reshape(seg_output,
                                  [batch_size * seq_length, width])
gathered_tensor = tf.gather(flat_sequence_tensor, flat_positions)

In [305]:
gathered_tensor

<tf.Tensor: shape=(160, 768), dtype=float32, numpy=
array([[ 0.05825796, -0.15728351,  0.03383664, ...,  0.21044591,
        -0.20374396,  0.01895263],
       [ 0.04630796, -0.35663825, -0.02367768, ...,  0.2625608 ,
        -0.26293308,  0.00389194],
       [ 0.02822801, -0.03218177,  0.09385708, ...,  0.10875528,
        -0.03896865,  0.0520727 ],
       ...,
       [ 0.03363404, -0.9710863 ,  0.1194402 , ...,  0.60961324,
         0.01628826, -0.10343488],
       [ 0.01762544, -0.1050678 , -0.0690262 , ...,  0.04092298,
        -0.02420309, -0.04515424],
       [ 0.01762544, -0.1050678 , -0.0690262 , ...,  0.04092298,
        -0.02420309, -0.04515424]], dtype=float32)>

In [313]:
seg_output

<tf.Tensor: shape=(4, 256, 768), dtype=float32, numpy=
array([[[ 0.02781948, -0.02764396,  0.09445661, ...,  0.10266508,
         -0.03675129,  0.05366931],
        [ 0.02822801, -0.03218177,  0.09385708, ...,  0.10875528,
         -0.03896865,  0.0520727 ],
        [ 0.02795909, -0.03357469,  0.09306844, ...,  0.11037768,
         -0.04007384,  0.05098892],
        ...,
        [ 0.45052794, -0.60000634,  0.5118584 , ...,  0.76278085,
         -0.37980098,  0.02797634],
        [ 0.47552156, -0.56079006,  0.47169533, ...,  0.7253324 ,
         -0.41338626, -0.02545325],
        [ 0.47995764, -0.39457887,  0.56018215, ...,  0.7901278 ,
         -0.5410525 ,  0.01307989]],

       [[ 0.02781946, -0.02764396,  0.09445661, ...,  0.10266499,
         -0.0367513 ,  0.05366932],
        [ 0.02822802, -0.03218175,  0.09385711, ...,  0.10875521,
         -0.03896853,  0.05207273],
        [ 0.02795907, -0.03357475,  0.09306846, ...,  0.11037759,
         -0.04007381,  0.05098896],
        ...,

In [315]:
seg_output[0][1]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([ 2.82280147e-02, -3.21817696e-02,  9.38570797e-02,  5.64307794e-02,
       -5.62167168e-02, -4.53090817e-02,  1.57649189e-01,  3.94005105e-02,
        2.54391372e-01,  3.69562387e-01,  8.11151266e-02,  8.10427666e-02,
        1.01128034e-01, -1.24113932e-01,  1.88320726e-02, -2.26577967e-02,
       -6.03011549e-02, -6.14533797e-02, -4.82205078e-02, -7.14849681e-03,
       -1.13110252e-01, -2.57278860e-01,  1.38709605e-01,  5.41408807e-02,
       -5.95906079e-01,  1.54145792e-01,  4.34713066e-03, -7.52546340e-02,
       -3.77097577e-02, -4.28135097e-02,  3.97741646e-02, -2.14224383e-02,
       -2.95710802e-01, -4.20654202e+00, -1.30429447e-01, -6.88063502e-02,
        1.29382089e-01,  1.59032136e-01, -1.14017189e-01,  4.67470199e-01,
       -1.06012955e-01,  4.08833838e+00,  9.22491401e-02,  7.57985413e-02,
       -3.10274288e-02,  4.93434966e-02, -3.73269320e-02,  1.56765908e-01,
       -1.74741969e-02, -2.23300606e-03, -5.87098598

In [309]:
masked_lm_positions[0]

<tf.Tensor: shape=(40,), dtype=int32, numpy=
array([21, 26,  1, 25, 38, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0])>

In [316]:
tf.reshape(gathered_tensor, (-1, 40, 768))[0][2]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([ 2.82280147e-02, -3.21817696e-02,  9.38570797e-02,  5.64307794e-02,
       -5.62167168e-02, -4.53090817e-02,  1.57649189e-01,  3.94005105e-02,
        2.54391372e-01,  3.69562387e-01,  8.11151266e-02,  8.10427666e-02,
        1.01128034e-01, -1.24113932e-01,  1.88320726e-02, -2.26577967e-02,
       -6.03011549e-02, -6.14533797e-02, -4.82205078e-02, -7.14849681e-03,
       -1.13110252e-01, -2.57278860e-01,  1.38709605e-01,  5.41408807e-02,
       -5.95906079e-01,  1.54145792e-01,  4.34713066e-03, -7.52546340e-02,
       -3.77097577e-02, -4.28135097e-02,  3.97741646e-02, -2.14224383e-02,
       -2.95710802e-01, -4.20654202e+00, -1.30429447e-01, -6.88063502e-02,
        1.29382089e-01,  1.59032136e-01, -1.14017189e-01,  4.67470199e-01,
       -1.06012955e-01,  4.08833838e+00,  9.22491401e-02,  7.57985413e-02,
       -3.10274288e-02,  4.93434966e-02, -3.73269320e-02,  1.56765908e-01,
       -1.74741969e-02, -2.23300606e-03, -5.87098598

In [123]:
tf.reshape(tf.matmul(gathered_tensor, layer_x.embedding_table, transpose_b = True), (4, 40, 32000))

<tf.Tensor: shape=(4, 40, 32000), dtype=float32, numpy=
array([[[-0.96651876,  0.35889095, -0.72566915, ..., -0.6659758 ,
         -0.8975039 ,  0.3739977 ],
        [-1.169315  ,  0.04444015, -0.70775807, ..., -0.7374146 ,
         -0.9624998 ,  0.32622477],
        [-1.4330778 , -0.4447086 , -0.7175031 , ..., -0.8573061 ,
         -0.96034586,  0.14076054],
        ...,
        [-0.20153634,  1.3292346 , -0.7900561 , ..., -0.13466144,
         -0.00956674,  0.3052761 ],
        [-0.20153634,  1.3292346 , -0.7900561 , ..., -0.13466144,
         -0.00956674,  0.3052761 ],
        [-0.20153634,  1.3292346 , -0.7900561 , ..., -0.13466144,
         -0.00956674,  0.3052761 ]],

       [[-0.20024347,  1.3415475 , -0.8412107 , ..., -0.16264868,
          0.02211582,  0.27375466],
        [-0.20024347,  1.3415475 , -0.84121084, ..., -0.16264874,
          0.02211584,  0.27375466],
        [-0.20024353,  1.3415475 , -0.84121084, ..., -0.16264856,
          0.02211585,  0.27375472],
        ...

In [119]:
d = layer_x.dense(gathered_tensor)
l = layer_x.layer_norm(d)
mm = tf.matmul(l, layer_x.embedding_table, transpose_b = True)

In [127]:
sum(d[0] - d[1]), sum(d[0]-d[2])

(<tf.Tensor: shape=(), dtype=float32, numpy=-53.845802>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-95.79878>)

In [129]:
sum(l[0] - l[1]), sum(l[0]-l[2])

(<tf.Tensor: shape=(), dtype=float32, numpy=0.18630481>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.4441756>)

In [132]:
layer_x.bias

<tf.Variable 'transform/bias:0' shape=(32000,) dtype=float32, numpy=
array([-0.3385531 , -0.27337646, -0.6101586 , ..., -0.06429594,
       -0.06523742, -0.06201189], dtype=float32)>

In [139]:
k = mm + layer_x.bias

In [140]:
logits = tf.reshape(k, shape = [ -1, 40, 32000])

In [96]:
target_value = np.zeros(32000)
target_value[12382]=1

In [111]:
tf.cast(target_value, dtype = tf.float32)[tf.newaxis, tf.newaxis, :]

<tf.Tensor: shape=(1, 1, 32000), dtype=float32, numpy=array([[[0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [142]:
logits_orig = layer_x([seg_output, masked_lm_positions])

In [163]:
tf.reduce_mean(tf.reduce_sum(tf.one_hot(inputs['masked_lm_ids'][0], depth = 32000) * logits_orig, axis = 2))

<tf.Tensor: shape=(), dtype=float32, numpy=-12.617973>

In [179]:
tf.reduce_sum(tf.one_hot(inputs['masked_lm_ids'][0], depth = 32000) * logits_orig[0], axis = 1)

<tf.Tensor: shape=(40,), dtype=float32, numpy=
array([ -7.9717956,  -9.810534 , -11.699598 , -11.021503 ,  -8.9260025,
        -8.948836 ,  -8.158123 , -10.64055  ,  -9.918285 , -10.824669 ,
        -7.0177984, -11.998983 ,  -3.5431592,  -5.5068183,  -2.327545 ,
        -3.5452194,  -8.443156 , -10.519745 ,  -8.423456 ,  -9.255592 ,
       -10.670185 , -12.049931 ,  -8.298307 , -17.980309 , -17.980309 ,
       -17.980309 , -17.980309 , -17.980309 , -17.980309 , -17.980309 ,
       -17.980309 , -17.980309 , -17.980309 , -17.980309 , -17.980309 ,
       -17.980309 , -17.980309 , -17.980309 , -17.980309 , -17.980309 ],
      dtype=float32)>

In [181]:
inputs['masked_lm_ids'][0][14]

<tf.Tensor: shape=(), dtype=int64, numpy=17>

In [166]:
tf.nn.softmax(logits[0])

<tf.Tensor: shape=(40, 32000), dtype=float32, numpy=
array([[1.48039021e-08, 1.49417119e-08, 1.48754555e-08, ...,
        6.09833160e-06, 8.42175996e-06, 3.68102610e-06],
       [1.48186654e-08, 1.46814578e-08, 1.45875463e-08, ...,
        5.94649600e-06, 8.69370433e-06, 4.28157273e-06],
       [1.57232360e-08, 1.46630192e-08, 1.49479167e-08, ...,
        5.22839855e-06, 8.96226993e-06, 5.39560051e-06],
       ...,
       [1.55328639e-08, 1.82006765e-08, 1.58578128e-08, ...,
        6.68268922e-06, 8.18753688e-06, 5.43581791e-06],
       [1.55328639e-08, 1.82006765e-08, 1.58578128e-08, ...,
        6.68268922e-06, 8.18753688e-06, 5.43581791e-06],
       [1.55328639e-08, 1.82006765e-08, 1.58578128e-08, ...,
        6.68268922e-06, 8.18753688e-06, 5.43581791e-06]], dtype=float32)>

In [173]:
scc = tf.keras.losses.sparse_categorical_crossentropy(inputs['masked_lm_ids'][0], logits[0], from_logits = True)

In [174]:
scc

<tf.Tensor: shape=(40,), dtype=float32, numpy=
array([ 7.971795 ,  9.810534 , 11.699598 , 11.021503 ,  8.9260025,
        8.948836 ,  8.158123 , 10.64055  ,  9.918285 , 10.824669 ,
        7.0177984, 11.998983 ,  3.5431592,  5.5068183,  2.327545 ,
        3.5452194,  8.443156 , 10.519745 ,  8.423456 ,  9.255592 ,
       10.670185 , 12.049931 ,  8.298307 , 17.980309 , 17.980309 ,
       17.980309 , 17.980309 , 17.980309 , 17.980309 , 17.980309 ,
       17.980309 , 17.980309 , 17.980309 , 17.980309 , 17.980309 ,
       17.980309 , 17.980309 , 17.980309 , 17.980309 , 17.980309 ],
      dtype=float32)>

In [169]:
scc = tf.keras.losses.sparse_categorical_crossentropy(inputs['masked_lm_ids'][0], tf.nn.softmax(logits[0]))

In [170]:
scc

<tf.Tensor: shape=(40,), dtype=float32, numpy=
array([ 7.9720297,  9.810738 , 11.699795 , 11.021713 ,  8.926208 ,
        8.949032 ,  8.158358 , 10.640753 ,  9.91852  , 10.824872 ,
        7.018003 , 11.999188 ,  3.543393 ,  5.507016 ,  2.3277202,
        3.545422 ,  8.443391 , 10.519949 ,  8.42366  ,  9.255796 ,
       10.67042  , 12.050126 ,  8.298504 , 16.118301 , 16.118301 ,
       16.118301 , 16.118301 , 16.118301 , 16.118301 , 16.118301 ,
       16.118301 , 16.118301 , 16.118301 , 16.118301 , 16.118301 ,
       16.118301 , 16.118301 , 16.118301 , 16.118301 , 16.118301 ],
      dtype=float32)>

In [73]:
lm_numerator_loss = scc * inputs['masked_lm_weights'][0]

In [75]:
lm_denomirator_loss = inputs['masked_lm_weights'][0]

In [76]:
lm_numerator_loss

<tf.Tensor: shape=(40,), dtype=float32, numpy=
array([ 7.971795 ,  9.810534 , 11.699598 , 11.021503 ,  8.9260025,
        8.948836 ,  8.158123 , 10.64055  ,  9.918285 , 10.824669 ,
        7.0177984, 11.998983 ,  3.543159 ,  5.506818 ,  2.3275452,
        3.5452194,  8.443156 , 10.519745 ,  8.423456 ,  9.255592 ,
       10.670185 , 12.049931 ,  8.298307 ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ],
      dtype=float32)>

In [77]:
lm_denomirator_loss

<tf.Tensor: shape=(40,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)>

In [183]:
lm_loss = tf.math.divide_no_nan(tf.reduce_sum(lm_numerator_loss), tf.reduce_sum(lm_denomirator_loss))

In [184]:
lm_loss

<tf.Tensor: shape=(), dtype=float32, numpy=8.674773>